# Building an $n$ Qubit Quantum Teleporter
#### By: Arjun Bhamra

### Imports

In [38]:
import qiskit as q
from qiskit import *
import numpy as np
IBMQ.save_account("85d62c1de28ec95ddd2e38b8ad23ea3bb797c5bd5a17dc816fb2767c5141745b25d9a3fbf866f09a3d24228ed05f2576f87a3eb2e289cbe3154c639c8d96c212")
from qiskit.visualization import plot_histogram

configrc.store_credentials:WARNING:2021-03-15 12:44:16,170: Credentials already present. Set overwrite=True to overwrite.


### Helper Functions

In [39]:
#creates the bell state
def bell_state(qc, a, b):
    qc.h(a)
    qc.cx(a, b)

#uses the bell measurement
def bell_measurement(qc, a, b):
    qc.cx(a, b)
    qc.h(a)

#the correction gates based on the results    
def q_c_correction(qc, a, c0, c1):
    qc.x(a).c_if(c0, 1)
    qc.z(a).c_if(c1, 1)
    
def check_same_state(alice, bob, psi):
    a = alice[0]
    b = [c for c in bob[0] if c != 0]
    print("Psi state: ", psi[0], psi[1])
    print("Alice's state: ", a[0], a[1])
    print("Bob's state: ", b[0], b[1])
    # For simplicity I am rounding and summing to check for equality
    if np.round(a[0]+a[1], 5) == np.round(b[0]+ b[1], 5):
        print("State Successfully Teleported!")
    else:
        print("Error: Bob did not get the right state!")   

### Circuit Function (Might Change)

In [40]:
#creating the actual circuit
#here, input_states is a list of all of the psis that Alice is trying to send to Bob
def teleportation_circuit(input_states):
    for state in input_states:
        qr = q.QuantumRegister(3, 'q')
        c0 = q.ClassicalRegister(1, 'c0')
        c1 = q.ClassicalRegister(1, 'c1')
        
        qc = q.QuantumCircuit(qr, c0, c1)
        
        init_gate = q.extensions.Initialize(psi)

        #ansatz/initial state
        qc.append(init_gate, [0])

        qc.barrier()

        qc.snapshot('1')

        qc.barrier()

        #create bell pairs
        qc.h(1)
        qc.cx(1, 2)

        qc.barrier()

        #alice's gates
        qc.cx(0, 1)
        qc.h(0)

        qc.barrier()

        #measurements
        qc.measure(0, c0)
        qc.measure(1, c1)

        qc.barrier()

        qc.x(2).c_if(c0, 1)
        qc.z(2).c_if(c1, 1)

        qc.barrier()

        qc.snapshot('2')
        
        simulator = Aer.get_backend('statevector_simulator')

        # job = execute(qc, simulator, shots=1000)
        job = q.execute(qc, simulator)

        # Grab results from the job
        result = job.result()
        snapshots = result.data()['snapshots']['statevector']

        init_state = snapshots['1']
        final_state = snapshots['2']
        
        check_same_state(init_state, final_state, psi) 

In [41]:
psi = np.array([0.5533920757991503+0.3043529040180291j, 0.6147796854942953+0.4724113234904887j])    
    
input_states = np.array([psi, psi])  
teleportation_circuit(input_states)

Psi state:  (0.5533920757991503+0.3043529040180291j) (0.6147796854942953+0.4724113234904887j)
Alice's state:  (0.5533920757991503+0.3043529040180291j) (0.6147796854942953+0.4724113234904887j)
Bob's state:  (0.5533920757991502+0.30435290401802906j) (0.6147796854942955+0.4724113234904887j)
State Successfully Teleported!
Psi state:  (0.5533920757991503+0.3043529040180291j) (0.6147796854942953+0.4724113234904887j)
Alice's state:  (0.5533920757991503+0.3043529040180291j) (0.6147796854942953+0.4724113234904887j)
Bob's state:  (0.5533920757991504+0.30435290401802917j) (0.6147796854942953+0.47241132349048853j)
State Successfully Teleported!
